In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torch.utils as utils
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.datasets as dsets
from torchvision import models
from torchvision.utils import save_image
#from torch.utils.tensorboard import SummaryWriter

#writer = SummaryWriter('runs/thesis06_miniIN_5w_1s')

from tqdm import tqdm

from flows import PlanarFlow
from utils import Binarize
from codes import Linear_flipout, Flatten, count_parameters, EfficientNet

from torchmeta.datasets import MiniImagenet
from torchmeta.transforms import Categorical, ClassSplitter, Rotation
from torchvision.transforms import Compose, Resize, ToTensor
from torchmeta.utils.data import BatchMetaDataLoader


#from __future__ import print_function
import argparse
import cv2
import matplotlib.pyplot as plt

import os
cur_dir = "C:/Users/KJH/OneDrive - skku.edu/KJH/Projects/2019winter_research"
#cur_dir = "C:/Users/KJH-Laptop/OneDrive - skku.edu/KJH/Projects/2019winter_research/"
os.chdir(cur_dir)
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import time
import copy
import random as rd

device = torch.device('cuda')

class net(nn.Module):
    def __init__(self, num_classes):
        super(net, self).__init__()
        self.input_dim = [3, 84, 84]
        self.num_classes = num_classes
        
        self.ctx = torch.hub.load('rwightman/gen-efficientnet-pytorch', 'efficientnet_b0', pretrained=True)
        self.ctx.classifier = nn.Identity()
        
        self.layer = nn.LSTM(input_size=1280, hidden_size=1280, num_layers=5, bias=True, batch_first = True).to(device)
        self.dec_mu = nn.ModuleList()
        self.dec_logvar = nn.ModuleList()
        
        self.num_params = [[200 * 200, 200], [200 * 200, 200], [200 * self.num_classes, self.num_classes]]
        
        for layer_size in self.num_params:
            self.dec_mu.append(
                nn.Sequential(
                    nn.ELU(),
                    nn.Linear(1280, 1280, bias = True),
                    nn.ELU(),
                    nn.Linear(1280, layer_size[0] + layer_size[1], bias = True),
                ).to(device))
            
            self.dec_logvar.append(
                nn.Sequential(
                    nn.ELU(),
                    nn.Linear(1280, 1280, bias = True),
                    nn.ELU(),
                    nn.Linear(1280, layer_size[0] + layer_size[1], bias = True),
                ).to(device))
        
        self.encoder = nn.Sequential(
            torch.hub.load('rwightman/gen-efficientnet-pytorch', 'efficientnet_b0', pretrained=True),
            Flatten(),            
            nn.Linear(1280, 200, bias = False),
            nn.ELU()
        )
        self.encoder[0].classifier = nn.Identity()
            
            
    def forward(self, input_train, label_train, input_test, label_test, adapt_lr, adapt_step = 1):
        ctx = self.ctx(input_train).view(input_train.shape[0], self.num_classes, 256)
        h = torch.stack([ctx[torch.where(label_train == x)].mean(dim = 0) for x in range(self.num_classes)], dim = 1)
        h = h.view(self.num_classes, 1, -1)
        
        params = nn.ParameterList()
        param_vals = []
        kld = torch.tensor(0., device=device)
        c = torch.empty([5, 1, 1280], requires_grad = False, device = device).fill_(0)
        x_train_init = self.encoder(input_train)
        x_train = x_train_init

        for ind, (dec_mu, dec_logvar) in enumerate(zip(self.dec_mu, self.dec_logvar)):
            x = torch.empty([1, 1, 1280], requires_grad=False, device = device).normal_(0, 1)
            f, (h, c) = self.layer(x, (h, c))
            
            mu = dec_mu(f.view(1, -1)).squeeze()
            logvar = dec_logvar(f.view(1, -1)).squeeze()
            params.append(nn.Parameter(torch.stack((mu, logvar), dim = 0), requires_grad = True))
        
        optimizer = optim.SGD(params, lr = adapt_lr)
        
        for step in range(adapt_step):
            x_train = x_train_init
            kld = torch.tensor(0., device=device)
            for ind, param in enumerate(params):
                param.retain_grad()

                weight_mu = param[0, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
                bias_mu = param[0, self.num_params[ind][0]:]

                weight_logvar = param[1, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
                bias_logvar = param[1, self.num_params[ind][0]:]

                weight_noise = torch.empty(weight_mu.shape, requires_grad = False, device = device).normal_(0,1)
                bias_noise = torch.empty(bias_mu.shape, requires_grad = False, device = device).normal_(0,1)
                in_sign = torch.empty(x_train.shape, requires_grad = False, device = device).uniform_(-1,1).sign()
                out_sign = torch.empty([x_train.shape[0], self.num_params[ind][1]], requires_grad = False, device = device).uniform_(-1,1).sign()

                x_train = torch.mm(x_train, weight_mu) + torch.mm(in_sign * x_train, weight_noise * weight_mu * weight_logvar.div(2).exp()) * out_sign
                x_train += (1 + bias_noise * bias_logvar.div(2).exp()) * bias_mu
                x_train = F.elu(x_train)

                kld += (mu.pow(2) - logvar + logvar.exp() - 1).mean()/2
                
            if step < adapt_step:
                optimizer.zero_grad()
                loss = F.cross_entropy(x_train, label_train) + 1e-6 * kld
                loss.backward(retain_graph = True)
                optimizer.step()
                
                
        x_test_init = self.encoder(input_test)
        x_test = x_test_init
        kld = torch.tensor(0., device=device)
        for ind, param in enumerate(params):
            param.retain_grad()

            weight_mu = param[0, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
            bias_mu = param[0, self.num_params[ind][0]:]

            weight_logvar = param[1, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
            bias_logvar = param[1, self.num_params[ind][0]:]

            weight_noise = torch.empty(weight_mu.shape, requires_grad = False, device = device).normal_(0,1)
            bias_noise = torch.empty(bias_mu.shape, requires_grad = False, device = device).normal_(0,1)
            in_sign = torch.empty(x_test.shape, requires_grad = False, device = device).uniform_(-1,1).sign()
            out_sign = torch.empty([x_test.shape[0], self.num_params[ind][1]], requires_grad = False, device = device).uniform_(-1,1).sign()

            x_test = torch.mm(x_test, weight_mu) + torch.mm(in_sign * x_test, weight_noise * weight_mu * weight_logvar.div(2).exp()) * out_sign
            x_test += (1 + bias_noise * bias_logvar.div(2).exp()) * bias_mu
            x_test = F.elu(x_test)

            kld += (mu.pow(2) - logvar + logvar.exp() - 1).mean()/2

        return x_test, kld

In [2]:
batch_size = 16
meta_trainset = MiniImagenet('./data/',
                   # Number of ways
                   num_classes_per_task=5,
                   # Resize the images to 28x28 and converts them to PyTorch tensors (from Torchvision)
                   transform=Compose([Resize(84), ToTensor()]), #Compose([Resize(28), ToTensor()]),
                   # Transform the labels to integers (e.g. ("Glagolitic/character01", "Sanskrit/character14", ...) to (0, 1, ...))
                   target_transform=Categorical(num_classes=5),
                   # Creates new virtual classes with rotated versions of the images (from Santoro et al., 2016)
                   class_augmentations=[Rotation([90, 180, 270])],
                   meta_train=True,
                   download=True)
meta_trainset = ClassSplitter(meta_trainset, shuffle=True, num_train_per_class=5, num_test_per_class=15)
meta_trainloader = BatchMetaDataLoader(meta_trainset, batch_size=batch_size, num_workers=0)

model = net(5).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 0.5, 5)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 75
    reg /= batch_size

    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

Using cache found in C:\Users\KJH/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master
Using cache found in C:\Users\KJH/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master


  0) loss = 1.553334, kld = 0.000829, acc = 0.334167, time = 14.420 sec
 10) loss = 1.276524, kld = 0.000617, acc = 0.499167, time = 13.579 sec
 20) loss = 1.242520, kld = 0.000469, acc = 0.500000, time = 13.578 sec
 30) loss = 1.250029, kld = 0.000363, acc = 0.490000, time = 14.449 sec
 40) loss = 1.109555, kld = 0.000285, acc = 0.534167, time = 13.953 sec
 50) loss = 1.168489, kld = 0.000229, acc = 0.535000, time = 13.596 sec
 60) loss = 1.164641, kld = 0.000184, acc = 0.520000, time = 14.005 sec
 70) loss = 1.122053, kld = 0.000151, acc = 0.550833, time = 13.929 sec
 80) loss = 1.062900, kld = 0.000126, acc = 0.580000, time = 13.898 sec
 90) loss = 1.029055, kld = 0.000105, acc = 0.608333, time = 13.887 sec
100) loss = 1.058869, kld = 0.000089, acc = 0.577500, time = 13.631 sec
110) loss = 1.060300, kld = 0.000077, acc = 0.579167, time = 13.680 sec
120) loss = 1.053128, kld = 0.000068, acc = 0.569167, time = 13.747 sec
130) loss = 0.927961, kld = 0.000060, acc = 0.632500, time = 13.

In [3]:
optimizer = optim.Adam(model.parameters(), lr=1e-5)

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 0.5, 5)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 75
    reg /= batch_size

    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

  0) loss = 0.872206, kld = 0.000035, acc = 0.649167, time = 14.060 sec
 10) loss = 0.855012, kld = 0.000034, acc = 0.659167, time = 14.436 sec
 20) loss = 0.974498, kld = 0.000034, acc = 0.595833, time = 13.792 sec
 30) loss = 0.909586, kld = 0.000034, acc = 0.640000, time = 13.735 sec
 40) loss = 0.870590, kld = 0.000034, acc = 0.640833, time = 13.898 sec
 50) loss = 0.848191, kld = 0.000034, acc = 0.660000, time = 13.899 sec
 60) loss = 0.888337, kld = 0.000035, acc = 0.651667, time = 13.887 sec
 70) loss = 0.836163, kld = 0.000034, acc = 0.672500, time = 13.813 sec
 80) loss = 0.878629, kld = 0.000034, acc = 0.650000, time = 13.750 sec
 90) loss = 0.837374, kld = 0.000035, acc = 0.667500, time = 13.626 sec
100) loss = 0.790566, kld = 0.000034, acc = 0.704167, time = 13.877 sec
110) loss = 0.934284, kld = 0.000034, acc = 0.598333, time = 13.582 sec
120) loss = 0.830715, kld = 0.000035, acc = 0.679167, time = 13.583 sec
130) loss = 0.774371, kld = 0.000035, acc = 0.708333, time = 13.

In [4]:
optimizer = optim.Adam(model.parameters(), lr=1e-7)

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 0.5, 5)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 75
    reg /= batch_size

    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

  0) loss = 0.821539, kld = 0.000034, acc = 0.670833, time = 13.924 sec
 10) loss = 0.929349, kld = 0.000033, acc = 0.621667, time = 14.017 sec
 20) loss = 0.786004, kld = 0.000034, acc = 0.690000, time = 13.828 sec
 30) loss = 0.882336, kld = 0.000034, acc = 0.653333, time = 14.142 sec
 40) loss = 0.810255, kld = 0.000034, acc = 0.668333, time = 13.902 sec
 50) loss = 0.883621, kld = 0.000033, acc = 0.636667, time = 14.057 sec
 60) loss = 1.028341, kld = 0.000034, acc = 0.583333, time = 13.781 sec
 70) loss = 0.822943, kld = 0.000034, acc = 0.667500, time = 13.576 sec
 80) loss = 0.864464, kld = 0.000033, acc = 0.659167, time = 13.716 sec
 90) loss = 0.779427, kld = 0.000034, acc = 0.680833, time = 13.956 sec
100) loss = 0.844642, kld = 0.000034, acc = 0.674167, time = 13.800 sec
110) loss = 0.791888, kld = 0.000034, acc = 0.685833, time = 13.644 sec
120) loss = 0.846007, kld = 0.000034, acc = 0.666667, time = 13.597 sec
130) loss = 0.804012, kld = 0.000034, acc = 0.680833, time = 13.

In [5]:
meta_testset  = MiniImagenet('./data/',
                   # Number of ways
                   num_classes_per_task=5,
                   # Resize the images to 28x28 and converts them to PyTorch tensors (from Torchvision)
                   transform=Compose([Resize(84), ToTensor()]),
                   # Transform the labels to integers (e.g. ("Glagolitic/character01", "Sanskrit/character14", ...) to (0, 1, ...))
                   target_transform=Categorical(num_classes=5),
                   # Creates new virtual classes with rotated versions of the images (from Santoro et al., 2016)
                   class_augmentations=[Rotation([90, 180, 270])],
                   meta_test=True,
                   download=True)
meta_testset = ClassSplitter(meta_testset, shuffle=True, num_train_per_class=5, num_test_per_class=15)
meta_testloader = BatchMetaDataLoader(meta_testset, batch_size=batch_size, num_workers=0)

tot_loss = torch.tensor(0., device=device)
tot_acc = torch.tensor(0., device=device)
tot_reg = torch.tensor(0., device=device)

for batch_idx, meta_test_batch in zip(range(num_batches), meta_testloader):
    start = time.time()
    train_inputs, train_targets = [x.to(device) for x in meta_test_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_test_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 0.5, 5)
        with torch.no_grad():            
            loss = criterion(pred, test_target)
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld

    tot_loss += cum_loss / batch_size
    tot_acc += accuracy  / (batch_size * 75)
    tot_reg += reg / batch_size
        
tot_loss /= num_batches
tot_acc /= num_batches
tot_reg /= num_batches
        
print("loss = %f, test_kld = %f, meta_test_acc = %f, time = %.3f sec" %(tot_loss, tot_reg, tot_acc, time.time() - start))

loss = 1.133484, test_kld = 0.000034, meta_test_acc = 0.521025, time = 6.855 sec


In [6]:
torch.save(model.state_dict(), "./save/thesis06_miniIN5w5s_5step")